In [1]:
import galsim
import piff
import numpy as np
import matplotlib.pyplot as plt
import pickle
import copy
import time
import cProfile
import pstats

print(piff.__version__)

1.4.0


In [2]:
# load data from a single ccd from subaru. Use the exact same input
# of what is coming in Piff in LSST stack (in meas_extensions_piff)

dic = pickle.load(open('piff_input_2024-06-24_13-43-22.pkl', 'rb'))

stars = dic['stars']
wcs = dic['wcs']
pointing = dic['pointing']
piffDefaultConfig = dic['piffConfig']

In [3]:
# run piff with current config in LSST stack
# using BasisPolynomial as interpolator of PSF parameters
# Same interpolator as in DES Y3

print(piffDefaultConfig)
logger = piff.config.setup_logger(verbose=2)
piffResult = piff.PSF.process(piffDefaultConfig)

def my_function():
    piffResult.fit(stars, wcs, pointing, logger=logger)

start = time.time()
cProfile.run('my_function()', 'profile_data')
end = time.time()

print('Total run time: ' + str(end - start))

p = pstats.Stats('profile_data')
p.strip_dirs().sort_stats('cumulative').print_stats(20)

Iteration 1: Fitting 118 stars


{'type': 'Simple', 'model': {'type': 'PixelGrid', 'scale': 0.16839226517851116, 'size': 25, 'interp': 'Lanczos(11)'}, 'interp': {'type': 'BasisPolynomial', 'order': 2}, 'outliers': {'type': 'Chisq', 'nsigma': 4.0, 'max_remove': 0.05}}


Beginning solution of matrix size (3750, 3750)
             Total chisq = 64075.13 / 72029 dof
Iteration 2: Fitting 118 stars
Beginning solution of matrix size (3750, 3750)
Checking 118 stars for outliers
Minimum dof = 497 with thresh = 627.063937 * 1.000000 = 627.063937
Maximum dof = 622 with thresh = 766.399073 * 1.000000 = 766.399073
Found 9 stars with chisq > thresh
             Removed 6 outliers
             Total chisq = 44329.60 / 68419 dof
Iteration 3: Fitting 112 stars
Beginning solution of matrix size (3750, 3750)
Checking 112 stars for outliers
Minimum dof = 497 with thresh = 627.063937 * 1.000000 = 627.063937
Maximum dof = 622 with thresh = 766.399073 * 1.000000 = 766.399073
Found 3 stars with chisq > thresh
             Removed 3 outliers
             Total chisq = 41877.24 / 66603 dof
Iteration 4: Fitting 109 stars
Beginning solution of matrix size (3750, 3750)
Checking 109 stars for outliers
Minimum dof = 497 with thresh = 627.063937 * 1.000000 = 627.063937
Maximum dof 

Total run time: 15.901031255722046
Mon Jul  1 16:53:17 2024    profile_data

         3116913 function calls (2970459 primitive calls) in 15.899 seconds

   Ordered by: cumulative time
   List reduced from 522 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   15.899   15.899 {built-in method builtins.exec}
        1    0.000    0.000   15.899   15.899 <string>:1(<module>)
        1    0.022    0.022   15.899   15.899 3874052704.py:9(my_function)
        1    0.142    0.142   15.877   15.877 psf.py:369(fit)
        4    0.021    0.005   14.803    3.701 simplepsf.py:163(single_iteration)
        4    0.017    0.004   13.257    3.314 basis_interp.py:96(solve)
        4   10.680    2.670   13.239    3.310 basis_interp.py:246(_solve_direct)
        4    1.468    0.367    1.468    0.367 _basic.py:52(solve)
      457    0.764    0.002    1.347    0.003 pixelgrid.py:226(chisq)
     2330    1.043    0.000    1.04

In [4]:
# run piff with current config in LSST stack
# except replace BasisPolynomial --> Polynomial

piffPolynomialConfig = copy.deepcopy(piffDefaultConfig)
piffPolynomialConfig['interp']['type'] = 'Polynomial'
print(piffPolynomialConfig)

logger = piff.config.setup_logger(verbose=2)
piffResult = piff.PSF.process(piffPolynomialConfig)

def my_function():
    piffResult.fit(stars, wcs, pointing, logger=logger)

start = time.time()
cProfile.run('my_function()', 'profile_data')
end = time.time()

print('Total run time: ' + str(end - start))

p = pstats.Stats('profile_data')
p.strip_dirs().sort_stats('cumulative').print_stats(20)

{'type': 'Simple', 'model': {'type': 'PixelGrid', 'scale': 0.16839226517851116, 'size': 25, 'interp': 'Lanczos(11)'}, 'interp': {'type': 'Polynomial', 'order': 2}, 'outliers': {'type': 'Chisq', 'nsigma': 4.0, 'max_remove': 0.05}}


Iteration 1: Fitting 118 stars
Fitting 625 parameter vectors using polynomial type poly with 2 positions
Checking 118 stars for outliers
Minimum dof = 497 with thresh = 627.063937 * 37.076431 = 23249.292993
Maximum dof = 622 with thresh = 766.399073 * 37.076431 = 28415.342582
Found 52 stars with chisq > thresh
             Removed 6 outliers
             Total chisq = 2398152.47 / 68325 dof
Iteration 2: Fitting 112 stars
Caught error singular matrix making params_var.  Setting all to 1.e100
Caught error singular matrix making params_var.  Setting all to 1.e100
Caught error singular matrix making params_var.  Setting all to 1.e100
Fitting 625 parameter vectors using polynomial type poly with 2 positions
Checking 112 stars for outliers
Minimum dof = 497 with thresh = 627.063937 * 44.536860 = 27927.458586
Maximum dof = 622 with thresh = 766.399073 * 44.536860 = 34133.007976
Found 61 stars with chisq > thresh
             Removed 6 outliers
             Total chisq = 2799765.51 / 64846 dof

Total run time: 920.2784969806671
Mon Jul  1 17:08:42 2024    profile_data

         80995847 function calls (79922311 primitive calls) in 920.273 seconds

   Ordered by: cumulative time
   List reduced from 567 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000  920.273  920.273 {built-in method builtins.exec}
        1    0.000    0.000  920.273  920.273 <string>:1(<module>)
        1    0.004    0.004  920.273  920.273 783620038.py:11(my_function)
        1    1.127    1.127  920.269  920.269 psf.py:369(fit)
       30    1.093    0.036  912.805   30.427 simplepsf.py:163(single_iteration)
     1675    2.428    0.001  886.803    0.529 pixelgrid.py:163(fit)
     1675  454.195    0.271  455.570    0.272 _basic.py:900(inv)
    17072  358.431    0.021  358.431    0.021 {method 'dot' of 'numpy.ndarray' objects}
     1675   63.676    0.038   63.715    0.038 _basic.py:1101(lstsq)
  2584867    0.603    0.000   20.